# SHAP with structured data regression

**Explainable AI with TensorFlow, Keras and SHAP**

SHAP (SHapley Additive exPlanations) is a game theoretic approach to explain the output of any machine learning model. It connects optimal credit allocation with local explanations using the classic Shapley values from game theory and their related extensions. [Learn more](https://shap.readthedocs.io/en/latest/index.html)  

*This is mainly based on the Keras tutorial ["Structured data classification from scratch"](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/) by François Chollet and ["Census income classification with Keras"](https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/neural_networks/Census%20income%20classification%20with%20Keras.html#Census-income-classification-with-Keras) by Scott Lundberg.*

## Setup

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import shap

tf.__version__

/Users/hendrikpfeifer/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'2.8.1'

In [2]:
# print the JS visualization code to the notebook
shap.initjs()

## Data

### Data import


- Let's download the data and load it into a Pandas dataframe:

In [3]:
df = pd.read_csv("car_prices_clean.csv", on_bad_lines="skip")
df = df.drop(columns=['Unnamed: 0', "seller"])

In [4]:
df.head()

,year,brand,model,type,state,condition,miles,color,interior,sellingprice
0,2015,kia,sorento,suv,ca,5.0,16639.0,white,black,21500
1,2015,kia,sorento,suv,ca,5.0,9393.0,white,beige,21500
2,2014,bmw,3 series,sedan,ca,4.5,1331.0,gray,black,30000
3,2015,volvo,s60,sedan,ca,4.1,14282.0,white,black,27750
4,2014,bmw,6 series gran coupe,sedan,ca,4.3,2641.0,gray,black,67000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533660 entries, 0 to 533659
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          533660 non-null  int64  
 1   brand         533660 non-null  object 
 2   model         533660 non-null  object 
 3   type          533660 non-null  object 
 4   state         533660 non-null  object 
 5   condition     533660 non-null  float64
 6   miles         533660 non-null  float64
 7   color         533660 non-null  object 
 8   interior      533660 non-null  object 
 9   sellingprice  533660 non-null  int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 40.7+ MB


### Data preparation

### Create labels and features

We need to encode our categorical features as one-hot numeric features (dummy variables):

In [5]:
dummies = pd.get_dummies(df[["brand", "model", "type", "state", "color", "interior"]])

In [6]:
dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533660 entries, 0 to 533659
Columns: 941 entries, brand_acura to interior_—
dtypes: uint8(941)
memory usage: 478.9 MB


In [7]:
print(dummies.head())


   brand_acura  brand_aston martin  brand_audi  brand_bentley  brand_bmw  \
0            0                   0           0              0          0   
1            0                   0           0              0          0   
2            0                   0           0              0          1   
3            0                   0           0              0          0   
4            0                   0           0              0          1   

   brand_buick  brand_cadillac  brand_chevrolet  brand_chrysler  brand_daewoo  \
0            0               0                0               0             0   
1            0               0                0               0             0   
2            0               0                0               0             0   
3            0               0                0               0             0   
4            0               0                0               0             0   

   ...  interior_green  interior_off-white  interior_ora

In [8]:
# make target variable

y = df['sellingprice']

In [9]:
X_numerical = df.drop(["sellingprice", "brand", "model", "type", "state", "color", "interior"], axis=1).astype('float64')

In [10]:
list_numerical = X_numerical.columns
list_numerical

Index(['year', 'condition', 'miles'], dtype='object')

In [11]:
# Create all features

X = pd.concat([X_numerical, dummies], axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533660 entries, 0 to 533659
Columns: 944 entries, year to interior_—
dtypes: float64(3), uint8(941)
memory usage: 491.1 MB


### Data splitting

- Let's split the data into a training and test set

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [13]:
X_train.head()

,year,condition,miles,brand_acura,brand_aston martin,brand_audi,brand_bentley,brand_bmw,brand_buick,brand_cadillac,...,interior_green,interior_off-white,interior_orange,interior_purple,interior_red,interior_silver,interior_tan,interior_white,interior_yellow,interior_—
228789,2014.0,3.8,29616.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
492477,2013.0,3.2,61597.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
108042,2012.0,4.7,11961.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146764,2011.0,3.8,37045.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
312891,2012.0,2.7,42300.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Feature preprocessing

In [14]:
scaler = StandardScaler().fit(X_train[list_numerical]) 

X_train[list_numerical] = scaler.transform(X_train[list_numerical])
X_test[list_numerical] = scaler.transform(X_test[list_numerical])

## Model

Now we can build the model using the Keras sequential API:

In [52]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu')
  ])

In [53]:
model.compile(optimizer="adam", 
              loss ="mse", 
              metrics=["mean_absolute_error"])

In [54]:
# will stop training when there is no improvement in 3 consecutive epochs

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [55]:
model.fit(X_train, y_train, 
         epochs=50,
         validation_data=(X_test, y_test), 
         callbacks=[callback]
         )

Epoch 1/50
11674/11674 [==============================] - 28s 2ms/step - loss: 199080304.0000 - mean_absolute_error: 10676.0166 - val_loss: 90575952.0000 - val_mean_absolute_error: 5835.3853
Epoch 2/50
11674/11674 [==============================] - 23s 2ms/step - loss: 55438492.0000 - mean_absolute_error: 4664.0195 - val_loss: 43717584.0000 - val_mean_absolute_error: 4205.1709
Epoch 3/50
11674/11674 [==============================] - 34s 3ms/step - loss: 37601804.0000 - mean_absolute_error: 3829.1743 - val_loss: 33786284.0000 - val_mean_absolute_error: 3498.4160
Epoch 4/50
11674/11674 [==============================] - 36s 3ms/step - loss: 29947962.0000 - mean_absolute_error: 3283.2161 - val_loss: 28252158.0000 - val_mean_absolute_error: 3110.1665
Epoch 5/50
11674/11674 [==============================] - 35s 3ms/step - loss: 25726818.0000 - mean_absolute_error: 2986.2852 - val_loss: 25178818.0000 - val_mean_absolute_error: 2895.8088
Epoch 6/50
11674/11674 [=============================

KeyboardInterrupt: 

- Configure the model with Keras Model.compile:

Let's visualize our connectivity graph:

In [47]:
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [23]:
loss, accuracy = model.evaluate(X_test, y_test)

print("MAE:", accuracy)

5004/5004 [==============================] - 4s 883us/step - loss: 8559516.0000 - mean_absolute_error: 1609.4264
MAE: 1609.4263916015625


## Perform inference

- The model you have developed can now classify a row from a CSV file directly after you've included the preprocessing layers inside the model itself. Let's demonstrate the process:

- Save the heart diseases classification model

In [24]:
model.save('shap_car_model')

2022-06-16 11:50:03.876302: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: shap_car_model/assets


- Load model

In [25]:
reloaded_model = tf.keras.models.load_model('shap_car_model')

In [37]:
predictions = reloaded_model.predict(X_train)

In [38]:
predictions

array([[33916.926, 33912.75 , 33917.348, ..., 33914.105, 33919.934,
        33911.082],
       [15486.537, 15484.985, 15486.724, ..., 15485.264, 15489.   ,
        15484.05 ],
       [23648.879, 23650.3  , 23651.275, ..., 23652.043, 23652.941,
        23648.645],
       ...,
       [22789.324, 22791.348, 22789.906, ..., 22792.588, 22788.988,
        22790.223],
       [17212.43 , 17211.6  , 17212.51 , ..., 17211.709, 17213.318,
        17211.027],
       [ 9916.016,  9915.749,  9916.983, ...,  9916.41 ,  9918.492,
         9913.793]], dtype=float32)

## SHAP

We use our model and a selection of 50 samples from the dataset to represent “typical” feature values (the so called background distribution).

In [40]:
explainer = shap.KernelExplainer(model, X_train.iloc[:50,:])

Now we use 500 perterbation samples to estimate the SHAP values for a given prediction (at index location 20). Note that this requires 500 * 50 evaluations of the model.

In [41]:
shap_values = explainer.shap_values(X_train.iloc[20,:], nsamples=500)

The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight

The so called force plot below shows how each feature contributes to push the model output from the base value (the average model output over the training dataset we passed) to the model output. Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue. To learn more about force plots, take a look at this [Nature BME paper](https://www.nature.com/articles/s41551-018-0304-0.epdf?author_access_token=vSPt7ryUfdSCv4qcyeEuCdRgN0jAjWel9jnR3ZoTv0PdqacSN9qNY_fC0jWkIQUd0L2zaj3bbIQEdrTqCczGWv2brU5rTJPxyss1N4yTIHpnSv5_nBVJoUbvejyvvjrGTb2odwWKT2Bfvl0ExQKhZw%3D%3D) from Lundberg et al. (2018).

In [42]:
shap.force_plot(explainer.expected_value[0], shap_values[0], X_train.iloc[20,:])

### Explain many predictions

If we take many force plot explanations such as the one shown above, rotate them 90 degrees, and then stack them horizontally, we can see explanations for an entire dataset (see content below). Here, we repeat the above explanation process for 50 individuals.

To understand how a single feature effects the output of the model we can plot the SHAP value of that feature vs. the value of the feature for all the examples in a dataset. Since SHAP values represent a feature's responsibility for a change in the model output, the plot below represents the change in the dependent variable. Vertical dispersion at a single value of represents interaction effects with other features. 

In [43]:
shap_values50 = explainer.shap_values(X_train.iloc[50:100,:], nsamples=500)

  0%|          | 0/50 [00:00<?, ?it/s]The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC()

In [44]:
shap.force_plot(explainer.expected_value[0], shap_values50[0], X_train.iloc[50:100,:])